In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.path import Path
from multiprocess import Pool

from scipy.spatial.kdtree import KDTree
from scipy.spatial.distance import cdist, pdist

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *
from cell_utilities import *
from learning_utilities import *

Setting environment for Gordon


No vtk
Cannot import mxnet.


In [2]:
n_orientation_bins = 10
orientation_bins = np.linspace(-np.pi/2, np.pi/2, n_orientation_bins+1)

size_bins = np.r_[np.linspace(0, 3000, 10), np.inf]

n_edge_length_bins = 10
edge_length_bins = np.r_[np.linspace(0, 100, n_edge_length_bins), np.inf]

n_edge_direction_bins = 10
edge_direction_bins = np.linspace(-np.pi/2, np.pi/2, n_edge_direction_bins+1)

In [3]:
neighborhood_radius_um = 50
neighborhood_radius = neighborhood_radius_um / XY_PIXEL_DISTANCE_LOSSLESS
print 'neighborhood_radius = %.2f' % neighborhood_radius

neighborhood_radius = 108.70


In [ ]:
n_radial = 4
# radial_bins = np.logspace(0, 2, 10, base=10)
radial_bins = np.linspace(0, 100, n_radial+1)

n_angular = 8
angular_bins = np.linspace(-np.pi, np.pi, n_angular+1)

In [ ]:
for stack in all_nissl_stacks:
    
    first_section, last_section = metadata_cache['section_limits'][stack]
    
    for sec in range(first_section, last_section+1):
#     for sec in [203]:

        sys.stderr.write('Section %d\n' % sec) # 5-9 seconds

        if is_invalid(stack=stack, sec=sec):
            continue
        
        cell_centroids = load_cell_data('centroid', stack, sec)
        
        t = time.time()

        tree = KDTree(cell_centroids)
        
        def get_neighbors(cell_ind):
            """
            Return neighbor cell indices sorted by distance.
            """
            neighbor_cell_indices = np.array(list(set(tree.query_ball_point(cell_centroids[cell_ind], r=neighborhood_radius)) - {cell_ind}))
            if len(neighbor_cell_indices) == 0:
                return None
            ngbr_vecs = cell_centroids[cell_ind] - cell_centroids[neighbor_cell_indices]
            neighbor_distances = np.sqrt(np.sum(ngbr_vecs**2, axis=1))
            order = np.argsort(neighbor_distances)
            return neighbor_cell_indices[order], ngbr_vecs[order]
                
        large_cell_indices = load_cell_data('largeCellIndices', stack=stack, sec=sec, ext='bp')

        pool = Pool(12)
        neighbor_info = pool.map(get_neighbors, large_cell_indices)
        pool.terminate()
        pool.join()

        sys.stderr.write('Neighbor search: %.2f seconds\n' % (time.time()-t)) # 5-9 seconds
        
        neighbor_cell_indices = {}
        neighbor_vectors = {}
        for cell_ind, ni in zip(large_cell_indices, neighbor_info):
            if ni is None:
                neighbor_cell_indices[cell_ind] = []
                neighbor_vectors[cell_ind] = []
            else:
                nci, nbr_vecs = ni
                neighbor_cell_indices[cell_ind] = nci
                neighbor_vectors[cell_ind] = nbr_vecs
    
        # Compute radial-angular histogram
        cell_orientations = load_cell_data('orientation', stack, sec)
        cell_orientations = np.array(map(normalize_angle, cell_orientations))

        t = time.time()
        
        radial_indices_all_cells = {}
        angular_indices_all_cells = {}
        for cell_ind, nbr_vecs in neighbor_vectors.iteritems():
            if len(nbr_vecs) == 0:
                radial_indices = []
                angular_indices = []
            else:
                radial_indices, angular_indices = \
                allocate_radial_angular_bins(nbr_vecs, cell_orientations[cell_ind],
                                            angular_bins=angular_bins, radial_bins=radial_bins)
            radial_indices_all_cells[cell_ind] = radial_indices
            angular_indices_all_cells[cell_ind] = angular_indices
        
        sys.stderr.write('Compute radial-angular histograms: %.2f seconds\n' % (time.time()-t)) # 1s
            
        # Dump to disk.
        fp = get_cell_data_filepath(what='neighborCellIndices', stack=stack, sec=sec, ext='hdf')
        save_hdf_v2(neighbor_cell_indices, fp)

        fp = get_cell_data_filepath(what='neighborVectors', stack=stack, sec=sec, ext='hdf')
        save_hdf_v2(neighbor_vectors, fp)

        fp = get_cell_data_filepath(what='neighborRadialHistBins', stack=stack, sec=sec, ext='hdf')
        save_hdf_v2(radial_indices_all_cells, fp)

        fp = get_cell_data_filepath(what='neighborAngularHistBins', stack=stack, sec=sec, ext='hdf')
        save_hdf_v2(angular_indices_all_cells, fp)

Neighbor search: 6.47 seconds
Compute radial-angular histograms: 0.88 seconds
/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/pandas/core/generic.py:1101: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->values] [items->None]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
Neighbor search: 6.21 seconds
Compute radial-angular histograms: 0.88 seconds
Neighbor search: 9.78 seconds
Compute radial-angular histograms: 1.34 seconds
Neighbor search: 6.41 seconds
Compute radial-angular histograms: 0.89 seconds
Neighbor search: 6.71 seconds
Compute radial-angular histograms: 0.96 seconds
Neighbor search: 6.43 seconds
Compute radial-angular histograms: 0.90 seconds
Neighbor search: 9.33 seconds
Compute radial-angular histograms: 1.30 seconds
Neighbor search: 6.46 seconds
Compute radial-angular histograms: 0.91 seconds
Neighbor search: 6.45 seconds
Com